In [46]:
import openai
import PyPDF2
import os
import re
import math
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import spacy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
import transformers
summarizer = transformers.pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", revision="a4f8f3e")
import warnings
warnings.simplefilter('ignore')
import json
from datetime import datetime
from library.exportation import export_prompt_response, export_article

In [2]:
# openAI API
# openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def load_pdf_text(pdf_path):
    """Load text from a PDF file."""
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text


def encode_chunks(chunks, nlp):
    """Encode text chunks using spaCy."""
    encoded_chunks = []
    for chunk in chunks:
        doc = nlp(chunk)
        encoded_chunks.append(doc.vector)  # Get the vector representation
    return encoded_chunks


def summarize_text(text, max_length=100):
    summarizer = transformers.pipeline("summarization")
    summary = summarizer(text, max_length=max_length, min_length=0, do_sample=False)
    return summary[0]['summary_text']

In [ ]:
file_name = '6055.HK'
folder_name = '3_data'
pdf_file_path = os.path.join(folder_name, f'{file_name}.pdf')
pdf_text = load_pdf_text(pdf_file_path)

In [4]:
# Initialize the RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,  # Adjust as needed
    chunk_overlap = 200  # Adjust as needed
)

# Split the text into chunks
text_chunks = text_splitter.split_text(pdf_text)

# Load the spaCy model
nlp = spacy.load("en_core_web_md")  # Load the spaCy model

# Encode the chunks from pdf
encoded_chunks = encode_chunks(text_chunks, nlp)

In [5]:
# prompt
json_file_path = r'prompt.json'

# Read and load the JSON data
with open(json_file_path, 'r') as file:
    prompts = json.load(file)

In [14]:
report = ''
c = 0
prompt_response = {}

for i, (k, q) in enumerate(prompts.items()):
    q_emb = nlp(q).vector
    # Find the most similar chunks to q_emb
    similarities = cosine_similarity([q_emb], encoded_chunks).flatten()
    top_idx = similarities.argsort()[::-1][0]
    top_text = text_chunks[top_idx]

    # use transformers to summarize the content 
    summarized_answer = summarize_text(top_text)
    
    report += summarized_answer + '\n'
    prompt_response[q] = summarized_answer

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

In [47]:
export_prompt_response(file_name, prompt_response)

JSON data has been exported to 1_prompt_log\6055.HK_20241120_0020.json


In [48]:
export_article(file_name, report)

report has been exported to 2_article_log\6055.HK_20241120_0020.txt
